In [ ]:
%load_ext autoreload
%autoreload 2
import glob

import pandas as pd
import numpy as np
import tqdm

from bokeh.models import HoverTool
from bokeh.io import output_notebook, output_file, reset_output, show, output_file
from bokeh.models import ColumnDataSource
from wellcomeml.viz.palettes import Wellcome11, WellcomeBackground
from bokeh.plotting import figure
from bokeh.models import HoverTool

output_notebook()

from grants_tagger.predict import predict_tags
from grants_tagger.utils import load_data

from datascience.grants import cleaning

## 1. Load grants file and tag with approach x-linear

We load the grants.csv file, which is the one with all grants (no filter), and then predict with xlinear.

In [ ]:
grants = pd.read_csv('../data/raw/grants.csv', index_col=0)
grants = cleaning.clean_grants(grants).fillna('')

predictions = pd.read_csv('../data/interim/mesh_pipeline_result.csv')
predictions.rename({'Grant id': 'Grant ID'}, axis=1, inplace=True)

merged_predictions_metadata = pd.merge(left=predictions, right=grants, how='left', on='Grant ID')
merged_predictions_metadata = merged_predictions_metadata[merged_predictions_metadata['Start Date'] > '2012']
merged_predictions_metadata[::-1].to_csv('../data/processed/merged_mesh_predictions_mesh_xlinear_for_validation.xlsx', index=None)

## 2. Check average tags per threshold

In [ ]:
results = []
for th in tqdm.tqdm(np.arange(0.0, 1, 0.001)):
    # Trims based on threshold
    subset = merged_predictions_metadata[merged_predictions_metadata['Prob'] >= th]
    
    subset = subset.groupby('Grant ID').agg({'Tag': 'count'})['Tag']
    
    avg = subset.mean()
    std = subset.std()
    median = subset.median()
    
    results.append(
        {'th': th, 'avg': avg, 'std': std, 'median': median, '95': subset.quantile(0.95), '05': subset.quantile(0.05)}
    )

## 3. Plot a nice graph

In [ ]:
output_file('../figures/threshold_per_average_html.html')

In [ ]:
p = figure(title="Average Tags per label", background_fill_color="#fafafa", width=800, height=400)

x = [r['th'] for r in results]
y = [r['avg'] for r in results]
yplus = [r['avg']+2*r['std'] for r in results]

yminus = [max(r['avg']-2*r['std'], 0) for r in results]
p.line(x, y, line_color=str(Wellcome11[0]), alpha=1)

p.varea(x, y1=yminus, y2=yplus, alpha=0.3)

p.background_fill_color = str(WellcomeBackground)
p.xaxis[0].ticker = np.arange(0.0, 1, 0.05)

p.add_tools(HoverTool(tooltips=[('x', '$x{0.0000f}'), ('y', '$y')], mode='vline'))

show(p)

## 4. Calculate precision-recall on publications

For the train-test split used during training, we calculate precision-recall. At this point, we're not yet using grants data.



In [ ]:
train_data_path = '../data/processed/train_mesh2021.jsonl'
test_data_path = '../data/processed/test_mesh2021.jsonl'

In [ ]:
X, y, _ = load_data(test_data_path)

In [ ]:
len(set([tag for tags in y for tag in tags]))

## 5. Load merged prediction files and explore it

In [ ]:
merged_predictions = pd.read_excel('../data/processed/merged_mesh_predictions_mesh_xlinear_for_validation.xlsx')

In [ ]:
merged_predictions['Tag'].unique().shape

In [ ]:
hist, bins = np.histogram(merged_predictions['Prob'], bins=150, density=True)

In [ ]:
p = figure(title="Probability density", background_fill_color="#fafafa", width=800, height=400)

x = bins
y = hist

p.vbar(x=x, top=y, line_color=str(Wellcome11[0]), width=0.005)

p.background_fill_color = str(WellcomeBackground)
p.xaxis[0].ticker = np.arange(0.0, 1, 0.05)

p.add_tools(HoverTool(tooltips=[('x', '$x{0.0000f}'), ('y', '$y')], mode='vline'))

show(p)

## 6. Load grants annotated by Wellcome

Now we load the many validation files generated by people at Wellcome and calculate "accuracy" metrics.

In [ ]:
%%time

dfs = []

for file in glob.glob('../data/raw/mesh_for_validation_2022/*.xlsx'):
    df = pd.read_excel(file)
    df.rename(columns={'0/1/2/?': 'Annotation', '0/1/2/3/?': 'Annotation'}, inplace=True)
    df.dropna(subset=['Annotation'], inplace=True)
    dfs.append(df[['Grant ID', 'MeSH Tag', 'MeSH Chapter', 'Prob', 'Annotation']])

In [ ]:
concatenated = pd.concat(dfs)

print(f"A total of {len(concatenated)} tags have been verified")
print(f"Unique annotations: {concatenated['Annotation'].unique()}")

In [ ]:
concatenated = concatenated[concatenated['Annotation'].isin([0, 1, 2, 3])]

In [ ]:
average_relevant = (concatenated['Annotation'] > 0).mean()
average_primary = (concatenated['Annotation'] == 1).mean()
average_secondary = (concatenated['Annotation'] >= 2).mean()

In [ ]:
print(f"Precision {average_relevant}. Primary: {average_primary}, Secondary: {average_secondary}")

In [ ]:
hist_pos, bins_pos = np.histogram(
    concatenated[(concatenated['Annotation'] > 0)]['Prob'], bins=50, density=True
)

hist_neg, bins_neg = np.histogram(
    concatenated[(concatenated['Annotation'] == 0)]['Prob'], bins=50, density=True
)

In [ ]:
output_file('density.html')
            
p = figure(title="Probability density", background_fill_color="#fafafa", width=800, height=400)


p.line(x=bins_pos[1:], y=hist_pos, line_color=str(Wellcome11[0]), width=2)
p.line(x=bins_neg[1:], y=hist_neg, line_color=str(Wellcome11[1]), width=2)

p.background_fill_color = str(WellcomeBackground)
p.xaxis[0].ticker = np.arange(0.0, 1, 0.05)

p.add_tools(HoverTool(tooltips=[('x', '$x{0.0000f}'), ('y', '$y')], mode='vline'))

show(p)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve

In [ ]:
fpr, tpr, th_roc = roc_curve(y_true=concatenated['Annotation'] > 0, y_score=concatenated['Prob'])
precision, recall, th_pr_rc = precision_recall_curve(y_true=concatenated['Annotation'] > 0, probas_pred=concatenated['Prob'])

In [ ]:
p = figure(title="With a 0.01 threshold, 80% of the tags are correct", background_fill_color="#fafafa", width=800, height=400)

source = {
    'precision': precision,
    'recall': recall,
    'th': th_pr_rc
}
p.line(x='recall', y='precision', source=source, line_color=str(Wellcome11[0]), width=2)

p.background_fill_color = str(WellcomeBackground)
p.xaxis[0].ticker = np.arange(0.0, 1, 0.05)

p.add_tools(HoverTool(tooltips=[('precision', '@precision'), ('recall', '@recall'), ('th', '@th')], mode='vline'))

p.xaxis.axis_label="% of tags output by the model"

p.yaxis.axis_label="% of correct tags"

show(p)

In [ ]:
for th in [0.1, 0.2, 0.5, 0.8, 0.9]:
    i = np.argmax(th < th_pr_rc)
    print(f"{th} {precision[i]:.2f} {recall[i]:.2f}")

In [ ]:
np.argmax(0.1 < th_pr_rc)

### 6. Precision recall per tree-level

In [ ]:
from datascience.mesh import load_mesh_tree

In [ ]:
string_to_id, id_to_string = load_mesh_tree(path_to_tree='../data/raw/desc2021.xml')

In [ ]:
string_to_min_tree_level = {
    string: min([len(idx.split('.')) for idx in mesh_ids]) for string, mesh_ids in string_to_id.items()
    if len(mesh_ids) > 0
}

In [ ]:
concatenated_level = []
for i in [1, 2, 3, 4, 5]:
    concatenated_level[i] = concatenated[concatenated

## 7. Explore research proposal predictions

We will answer the questions:

1. How many grants have "Research Question" ?

Only very few. (7063)

2. How many predictions differ by considering "Research Question" ?

In [ ]:
research_question_predictions = pd.read_csv('../data/interim/mesh_pipeline_result_research_question.csv')
standard_fields_predictions = pd.read_csv('../data/interim/mesh_pipeline_result.csv')

In [ ]:
research_question_predictions['Grant id'].unique().shape

In [ ]:
ids_that_have_research_question = research_question_predictions['Grant id'].unique()

In [ ]:
tags_standard_prediction = standard_fields_predictions[standard_fields_predictions['Grant id'].isin(ids_that_have_research_question)].groupby('Grant id').agg(
    {'Tag': set}
)

tags_research_question = research_question_predictions[research_question_predictions['Grant id'].isin(ids_that_have_research_question)].groupby('Grant id').agg(
    {'Tag': set}
)

In [ ]:
comparison_df = tags_standard_prediction.join(tags_research_question, on='Grant id', lsuffix='_standard', rsuffix='_rq')

In [ ]:
set({1,2}).intersection

In [ ]:
iou = comparison_df[['Tag_standard', 'Tag_rq']].apply(
    lambda x: len(x['Tag_standard'].union(x['Tag_rq']))-len(x['Tag_standard']), axis=1
)

In [ ]:
iou.mean()